## Batchmaker tutorial

This notebook is a means to showcase batchmaker's most basic functionality in its current implementation. As a test case, a first-level SPM-readable batchfile will be constructed with a canonical block design. Test and control conditions will alternate, each block lasting 25 seconds.

In [20]:
import pandas as pd 
import numpy as np 

from batchmaker import *

### Creating mock data

In the following cell, mock data for this test case will be constructed, namely file paths to putative .nii scans and regressors for the canonical block design.

In [21]:
"""create mock data"""

"""image paths"""

# scans
n_subj = 20
n_scans = 400 
paths = np.empty((n_subj,), dtype = np.object) # scans
mult_regs = np.empty((n_subj), dtype = np.object) # movement parameters

for i in range(n_subj):
    
    paths[i] = ["subj{}_scan{}.nii".format(str(i).zfill(2),
                str(j).zfill(3)) for j in range(n_scans)] # 400 scans per subject
    mult_regs[i] = "mult_reg_subj{}.txt".format(str(i).zfill(2)) # one mult_reg.txt file per subject
    

"""canonical block design"""

# creating names for the regressors
names = ['test_condition', 'control_condition']

# creating all durations and onsets
durations = np.array([25] * 32)
onsets_all = np.append(np.array(0), np.cumsum(durations)[:-1])

# splitting them into test and control 
durations_test, durations_control = np.array_split(durations, 2) 

onsets_test = onsets_all[[i for i in range(0, len(onsets_all), 2)]]
onsets_control = onsets_all[[i for i in range(1, len(onsets_all), 2)]]

### Batchmaker magic
Once the conditions and scans are ready, a first-level batch file can be constructed and exported in a few lines of python code using the batchmaker module. If one wishes to use values differing from the default parameters, this is easily achieved by adding an additional argument in the class calls of Conditions, Session, Batchfile or TimingParams. Additional conditions and sessions are automatically created with increasing size of the input arrays.

In [22]:
# make the conditions
cond = Conditions(names = names, onsets = [onsets_test, onsets_control], 
                  durations = [durations_test, durations_control])

# Make the sessions
for i in range(n_subj):
    mult_reg = mult_regs[i]
    sess = Session(np.array([paths[i]]), np.array([cond.mat]), mult_reg)

# Make the batchfile
batch = Batchfile('home/funkx', TimingParams(), sess)
batch.export('subX')